In [58]:
# !/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import pickle
#import plotly.express as px
#import plotly.figure_factory as ff
#import shap
import datetime
import logging
import yaml
from sklearn.impute import KNNImputer
from sklearn.metrics import recall_score, precision_score #, accuracy_score
# from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix
# from sklearn.metrics import jaccard_score, roc_auc_score, average_precision_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold, cross_validate #, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
# from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from category_encoders.count import CountEncoder


In [2]:
from training_functions import *

In [3]:
seed = 5000
ratio_balance = 0.9
k_folds = 5
verbose = 10

In [70]:
path='../outputs/'

In [4]:
df_train = get_data(name_sav='wA_train.sav', path='../outputs/')
df_test = get_data(name_sav='wA_test.sav', path='../outputs/')

Se hará una pequeña competencia entre un modelo clásico de clasificación, es decir, `RandomForest` y un modelo más comnplejo como `HistogramGradientBoosting`.

In [5]:
features_names =  ['Location',
                   'MinTemp',
                   'Rainfall',
                   'WindGustDir',
                   'WindGustSpeed',
                   'WindDir9am',
                   'WindDir3pm',
                   'WindSpeed9am',
                   'WindSpeed3pm',
                   'Humidity9am',
                   'Humidity3pm',
                   'Pressure9am',
                   'RainToday',
                   'month', 
                   'week_of_year']
objective_name = 'RainTomorrow'

In [20]:
model_name = 'RandomForestClassifier'
original_data = df_train.copy()
data = original_data[features_names]
float_names = list(data.select_dtypes(include='float64').columns)
categorical_names = list(data.select_dtypes(include='object').columns)
feature_names_order = get_feature_names_order(model_name=model_name,
                                              float_names=float_names,
                                              categorical_names=categorical_names)
# features = data[feature_names_order]
# label = original_data[objective_name]
X_train= data[feature_names_order]
y_train = original_data[objective_name]

In [17]:
X_test= df_test[feature_names_order]
y_test = df_test[objective_name]

In [32]:
X_train

,MinTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,month,week_of_year
0,13.4,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,Albury,W,W,WNW,No,12,49
1,7.4,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,Albury,WNW,NNW,WSW,No,12,49
2,12.9,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,Albury,WSW,W,WSW,No,12,49
3,9.2,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,Albury,NE,SE,E,No,12,49
4,17.5,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,Albury,W,ENE,NW,No,12,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141652,20.5,0.0,52.0,35.0,20.0,23.0,12.0,1013.2,Uluru,E,ESE,E,No,12,52
141653,18.0,0.0,54.0,30.0,31.0,17.0,7.0,1014.7,Uluru,ESE,E,ESE,No,12,53
141654,17.5,0.0,56.0,33.0,22.0,12.0,7.0,1012.6,Uluru,E,E,SE,No,12,53
141655,20.0,0.0,59.0,20.0,17.0,12.0,12.0,1007.2,Uluru,E,E,SSE,No,12,53


In [33]:
X_test

,MinTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,month,week_of_year
2474,20.4,0.0,54.0,0.0,7.0,46.0,17.0,1013.4,Albury,ENE,Unidentified,ESE,No,1,53
2475,20.9,0.4,50.0,9.0,17.0,54.0,30.0,1011.1,Albury,SSE,SSE,SE,No,1,53
2476,18.4,2.2,48.0,11.0,39.0,62.0,67.0,1014.0,Albury,ENE,ESE,ENE,Yes,1,53
2477,17.3,15.6,39.0,9.0,17.0,74.0,65.0,1017.9,Albury,SSE,SE,SSE,Yes,1,1
2478,15.5,6.8,31.0,6.0,9.0,92.0,63.0,1016.3,Albury,ENE,SE,SSE,Yes,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142185,3.5,0.0,31.0,15.0,13.0,59.0,27.0,1024.7,Uluru,E,ESE,E,No,6,25
142186,2.8,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,Uluru,E,SE,ENE,No,6,25
142187,3.6,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,Uluru,NNW,SE,N,No,6,25
142188,5.4,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,Uluru,N,SE,WNW,No,6,25


In [7]:
float_names

['MinTemp',
 'Rainfall',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am']

In [8]:
categorical_names

['Location',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday',
 'month',
 'week_of_year']

In [9]:
 data[features_names].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116217 entries, 0 to 141656
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       116217 non-null  object 
 1   MinTemp        115740 non-null  float64
 2   Rainfall       115075 non-null  float64
 3   WindGustDir    116217 non-null  object 
 4   WindGustSpeed  108408 non-null  float64
 5   WindDir9am     116217 non-null  object 
 6   WindDir3pm     116217 non-null  object 
 7   WindSpeed9am   115002 non-null  float64
 8   WindSpeed3pm   114707 non-null  float64
 9   Humidity9am    114731 non-null  float64
 10  Humidity3pm    114363 non-null  float64
 11  Pressure9am    104883 non-null  float64
 12  RainToday      116217 non-null  object 
 13  month          116217 non-null  object 
 14  week_of_year   116217 non-null  object 
dtypes: float64(8), object(7)
memory usage: 14.2+ MB


# RANDOM FOREST

In [10]:
numeric_transformer = Pipeline(
    steps=[('imputer', KNNImputer(n_neighbors=5, weights="uniform"))])
categorical_transformer = Pipeline(
    steps=[('CountEncoder', CountEncoder(normalize=True))])
preprocessor = ColumnTransformer(remainder='passthrough',
                                 transformers=[('numeric', numeric_transformer, float_names),
                                               ('categorical', categorical_transformer, categorical_names)])

In [12]:
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(
                                random_state=seed, sampling_strategy=ratio_balance)),
                            ("estimator",  RandomForestClassifier())])

In [13]:
clf = transform.fit(X_train, y_train)

In [15]:
clf.score(X_train, y_train)

0.905977610848671

In [25]:
y_pred = clf.predict(X_test)

In [26]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[16262,  3765],
       [ 1554,  4392]])

In [27]:
recall = recall_score(y_test, y_pred)
recall

0.7386478304742684

In [28]:
precision = precision_score(y_test, y_pred)
precision

0.5384332475174697

In [29]:
f_1 = f1_score(y_test, y_pred)
f_1

0.6228462029355457

In [30]:
m_c = matthews_corrcoef(y_test, y_pred)
m_c

0.49845619006285363

# HISTOGRAM GRADIENT BOOSTING

In [44]:
model_name = 'HistGradientBoostingClassifier'
original_data = df_train.copy()
data = original_data[features_names]
float_names = list(data.select_dtypes(include='float64').columns)
categorical_names = list(data.select_dtypes(include='object').columns)
feature_names_order = get_feature_names_order(model_name=model_name,
                                              float_names=float_names,
                                              categorical_names=categorical_names)
# features = data[feature_names_order]
# label = original_data[objective_name]
X_train= data[feature_names_order]
y_train = original_data[objective_name]

In [45]:
X_test= df_test[feature_names_order]
y_test = df_test[objective_name]

In [46]:
X_train

,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,month,week_of_year,MinTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am
0,Albury,W,W,WNW,No,12,49,13.4,0.6,44.0,20.0,24.0,71.0,22.0,1007.7
1,Albury,WNW,NNW,WSW,No,12,49,7.4,0.0,44.0,4.0,22.0,44.0,25.0,1010.6
2,Albury,WSW,W,WSW,No,12,49,12.9,0.0,46.0,19.0,26.0,38.0,30.0,1007.6
3,Albury,NE,SE,E,No,12,49,9.2,0.0,24.0,11.0,9.0,45.0,16.0,1017.6
4,Albury,W,ENE,NW,No,12,49,17.5,1.0,41.0,7.0,20.0,82.0,33.0,1010.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141652,Uluru,E,ESE,E,No,12,52,20.5,0.0,52.0,35.0,20.0,23.0,12.0,1013.2
141653,Uluru,ESE,E,ESE,No,12,53,18.0,0.0,54.0,30.0,31.0,17.0,7.0,1014.7
141654,Uluru,E,E,SE,No,12,53,17.5,0.0,56.0,33.0,22.0,12.0,7.0,1012.6
141655,Uluru,E,E,SSE,No,12,53,20.0,0.0,59.0,20.0,17.0,12.0,12.0,1007.2


In [47]:
X_test

,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,month,week_of_year,MinTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am
2474,Albury,ENE,Unidentified,ESE,No,1,53,20.4,0.0,54.0,0.0,7.0,46.0,17.0,1013.4
2475,Albury,SSE,SSE,SE,No,1,53,20.9,0.4,50.0,9.0,17.0,54.0,30.0,1011.1
2476,Albury,ENE,ESE,ENE,Yes,1,53,18.4,2.2,48.0,11.0,39.0,62.0,67.0,1014.0
2477,Albury,SSE,SE,SSE,Yes,1,1,17.3,15.6,39.0,9.0,17.0,74.0,65.0,1017.9
2478,Albury,ENE,SE,SSE,Yes,1,1,15.5,6.8,31.0,6.0,9.0,92.0,63.0,1016.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142185,Uluru,E,ESE,E,No,6,25,3.5,0.0,31.0,15.0,13.0,59.0,27.0,1024.7
142186,Uluru,E,SE,ENE,No,6,25,2.8,0.0,31.0,13.0,11.0,51.0,24.0,1024.6
142187,Uluru,NNW,SE,N,No,6,25,3.6,0.0,22.0,13.0,9.0,56.0,21.0,1023.5
142188,Uluru,N,SE,WNW,No,6,25,5.4,0.0,37.0,9.0,9.0,53.0,24.0,1021.0


In [48]:
#numeric_transformer = Pipeline(
#    steps=[('imputer', KNNImputer(n_neighbors=5, weights="uniform"))])
categorical_transformer = Pipeline(
    steps=[('CountEncoder', CountEncoder(normalize=True))])
preprocessor = ColumnTransformer(remainder='passthrough',
                                 transformers=[('categorical', categorical_transformer, categorical_names)])

In [49]:
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(
                                random_state=seed, sampling_strategy=ratio_balance)),
                            ("estimator",  HistGradientBoostingClassifier())])

In [50]:
clf = transform.fit(X_train, y_train)

In [51]:
y_pred = clf.predict(X_test)

In [52]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[16279,  3748],
       [ 1468,  4478]])

In [53]:
recall = recall_score(y_test, y_pred)
recall

0.7531113353514968

In [54]:
precision = precision_score(y_test, y_pred)
precision

0.5443715049841964

In [55]:
f_1 = f1_score(y_test, y_pred)
f_1

0.6319503245836862

In [56]:
m_c = matthews_corrcoef(y_test, y_pred)
m_c

0.5111547128874186

# CROSS-VALIDATION DEL MODELO ELEGIDO SIN OPTIMIZAR

In [59]:
transform

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  Pipeline(steps=[('CountEncoder',
                                                                   CountEncoder(combine_min_nan_groups=True,
                                                                                normalize=True))]),
                                                  ['Location', 'WindGustDir',
                                                   'WindDir9am', 'WindDir3pm',
                                                   'RainToday', 'month',
                                                   'week_of_year'])])),
                ('RandomUnderSampler',
                 RandomUnderSampler(random_state=5000, sampling_strategy=0.9)),
                ('estimator', HistGradientBoostingClassifier())])

In [60]:
c_v = StratifiedKFold(n_splits=k_folds,
                      shuffle=True,
                      random_state=seed)

In [61]:
scores = {'f1': 'f1',
          'precision': 'precision',
          'recall': 'recall',
          'm_c': make_scorer(matthews_corrcoef)}

In [62]:
cv_results = cross_validate(transform,
                            X_train,
                            y_train,
                            cv=c_v,
                            scoring=scores,
                            verbose=10)

[CV] START .....................................................................
[CV] END  f1: (test=0.639) m_c: (test=0.526) precision: (test=0.555) recall: (test=0.755) total time=   1.5s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    1.5s


[CV] END  f1: (test=0.637) m_c: (test=0.522) precision: (test=0.553) recall: (test=0.751) total time=   1.5s
[CV] START .....................................................................
[CV] END  f1: (test=0.644) m_c: (test=0.533) precision: (test=0.554) recall: (test=0.770) total time=   1.4s
[CV] START .....................................................................
[CV] END  f1: (test=0.643) m_c: (test=0.531) precision: (test=0.558) recall: (test=0.760) total time=   1.4s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    5.8s


[CV] END  f1: (test=0.638) m_c: (test=0.524) precision: (test=0.552) recall: (test=0.757) total time=   1.4s


In [63]:
cv_results

{'fit_time': array([1.37882662, 1.42404819, 1.23855615, 1.26180696, 1.24595594]),
 'score_time': array([0.11588192, 0.11280656, 0.11290693, 0.11363268, 0.11292124]),
 'test_f1': array([0.63939245, 0.63689162, 0.64417723, 0.64327676, 0.63821138]),
 'test_precision': array([0.55453258, 0.55306151, 0.55384402, 0.55756719, 0.55172899]),
 'test_recall': array([0.75491708, 0.75067489, 0.76972035, 0.76012341, 0.75684535]),
 'test_m_c': array([0.52577452, 0.52232296, 0.53258575, 0.53112148, 0.52418493])}

In [66]:
round(cv_results['test_f1'].mean(), 2)

0.64

In [67]:
round(cv_results['test_precision'].mean(), 2)

0.55

In [68]:
round(cv_results['test_recall'].mean(), 2)

0.76

In [69]:
round(cv_results['test_m_c'].mean(), 2)

0.53

# GUARDADO DE MODELO

In [71]:
pickle.dump(clf, open(f'{path}rain_model.sav', 'wb'))

# Experimentos ###################

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
X, y = make_classification(n_classes=2, class_sep=2,
 weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

In [ ]:
serie = pd.Series(y)
serie.value_counts()

In [ ]:
rus = RandomUnderSampler(random_state=42, sampling_strategy=0.8)
X_res, y_res = rus.fit_resample(X, y)

In [ ]:
s = pd.Series(y_res)
s.value_counts()

In [ ]:
125*(0.8)